적용해볼거

1) aug_v2 o
2) Mecab 
3) Epoch 늘리기 => 20 x
4) 다른 토크나이저 사용
5) 

mecab = Mecab()
data['sentence'] = data['sentence'].apply(lambda x : ' '.join(mecab.morphs(x))

In [1]:
import numpy as np
import os
import torch

def seed_everything(seed: int = 42):
    #random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(0)

In [2]:
#baseline 코드에서 가져온 함수들
import pickle as pickle
import pandas as pd
from sklearn.metrics import accuracy_score

# Dataset 구성.
class RE_Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
# 처음 불러온 tsv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다.
# 변경한 DataFrame 형태는 baseline code description 이미지를 참고해주세요.
def preprocessing_dataset(dataset, label_type):
    label = []
    for i in dataset[8]:
        if i == 'blind': label.append(100)
        else: label.append(label_type[i])
    out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
    return out_dataset

# 평가를 위한 metrics function.
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
    }

# tsv 파일을 불러옵니다.
def load_data(dataset_dir):
    # load label_type, classes
    with open('input/data/label_type.pkl', 'rb') as f:
        label_type = pickle.load(f)
    # load dataset
    dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
    # preprecessing dataset
    dataset = preprocessing_dataset(dataset, label_type)
  
    return dataset

# bert input을 위한 tokenizing.
# tip! 다양한 종류의 tokenizer와 special token들을 활용하는 것으로도 새로운 시도를 해볼 수 있습니다.
# baseline code에서는 2가지 부분을 활용했습니다.
def tokenized_dataset(dataset, tokenizer):
    concat_entity = []
    for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
          concat_entity,
          list(dataset['sentence']),
          return_tensors="pt",
          padding=True,
          truncation=True,
          max_length=128,
          add_special_tokens=True,
    )
    return tokenized_sentences

# 각종 데이터 로드
# load dataset
#train_dataset = load_data("input/data/train/train.tsv")
train_dataset = load_data("input/data/train/labeled_train_aug_fix.tsv")
#dev_dataset = load_data("./dataset/train/train_dev.tsv")
train_label = train_dataset['label'].values
#dev_label = dev_dataset['label'].values

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [7]:
#https://github.com/monologg/KoELECTRA
from transformers import AutoTokenizer, ElectraConfig, ElectraForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, XLMRobertaConfig
import warnings
warnings.filterwarnings('ignore')
#model_name = 'monologg/koelectra-base-v3-discriminator'
model_name = "xlm-roberta-large"
#model_name = 'monologg/koelectra-small-discriminator'
#model_name = 'monologg/koelectra-base-discriminator'

# tokenizing dataset
tokenizer_v3 = AutoTokenizer.from_pretrained(model_name)
#tokenizer_v3.add_special_tokens({"additional_special_tokens" : ["[E1]", "[E2]"]})
#tokenizer_v3 = XLMRobertaTokenizer.from_pretrained(model_name)
tokenized_train = tokenized_dataset(train_dataset, tokenizer_v3)
#tokenized_dev = tokenized_dataset(dev_dataset, tokenizer_v3)

# make dataset for pytorch.
RE_train_dataset = RE_Dataset(tokenized_train, train_label)
#RE_dev_dataset = RE_Dataset(tokenized_dev, dev_label)

#v3_config = ElectraConfig.from_pretrained(model_name)
v3_config = XLMRobertaConfig.from_pretrained(model_name)
v3_config.num_labels = 42
#model_v3 = ElectraForSequenceClassification.from_pretrained(model_name, config = v3_config)
model_v3 = XLMRobertaForSequenceClassification.from_pretrained('unlabeled_128/checkpoint-12700', config=v3_config)
model_v3.parameters
model_v3.resize_token_embeddings(len(tokenizer_v3))
model_v3.to(device)

We need to remove 14 to truncate the inputbut the first sequence has a length 11. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 93 to truncate the inputbut the first sequence has a length 9. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 154 to truncate the inputbut the first sequence has a length 12. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 36 to truncate the inputbut the first sequence has a length 8. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 14 to truncate the inputbut the first sequence has a length 11. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (Layer

In [8]:
from transformers import Trainer, TrainingArguments

# 사용한 option 외에도 다양한 option들이 있습니다.
# https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments 참고해주세요.
training_args = TrainingArguments(
        output_dir='./unlabeled_128',          # output directory
        save_total_limit=2,              # number of total save model.
        save_steps=100,                 # model saving step.
        num_train_epochs=1,              # total number of training epochs
        learning_rate=1e-5,               # learning_rate
        per_device_train_batch_size=8,  # batch size per device during training
        #per_device_eval_batch_size=16,   # batch size for evaluation
        warmup_steps=300,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=100,              # log saving step.
        #evaluation_strategy='steps', # evaluation strategy to adopt during training
                                    # `no`: No evaluation during training.
                                    # `steps`: Evaluate every `eval_steps`.
                                    # `epoch`: Evaluate every end of epoch.
        #eval_steps = 500,            # evaluation step.
)
trainer = Trainer(
        model=model_v3,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=RE_train_dataset,         # training dataset
        #eval_dataset=RE_dev_dataset,             # evaluation dataset
        compute_metrics=compute_metrics         # define metrics function
)

trainer.train()

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4060 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 68.00 MiB (GPU 0; 23.88 GiB total capacity; 22.82 GiB already allocated; 10.88 MiB free; 23.28 GiB reserved in total by PyTorch)

In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer,  ElectraConfig, ElectraForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, XLMRobertaConfig


def inference(model, tokenized_sent, device):
    dataloader = DataLoader(tokenized_sent, batch_size=8, shuffle=False)
    model.eval()
    output_pred = []
  
    for i, data in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            #print(data)
            outputs = model(
              input_ids=data['input_ids'].to(device),
              attention_mask=data['attention_mask'].to(device),
              #token_type_ids=data['token_type_ids'].to(device)
            )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result)
    
    return np.array(output_pred).flatten()

def load_test_dataset(dataset_dir, tokenizer):
    test_dataset = load_data(dataset_dir)
    test_label = test_dataset['label'].values
    # tokenizing dataset
    tokenized_test = tokenized_dataset(test_dataset, tokenizer)
    return tokenized_test, test_label

# load tokenizer
TOK_NAME = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(TOK_NAME)
tokenizer.add_special_tokens({"additional_special_tokens" : ["[E1]", "[E2]"]})

# load my model
MODEL_NAME ='labeled_128/checkpoint-8100'
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME)
model.parameters
model.to(device)

# load test datset
test_dataset_dir = "input/data/test/labeled_test.tsv"
test_dataset, test_label = load_test_dataset(test_dataset_dir, tokenizer)
test_dataset = RE_Dataset(test_dataset ,test_label)

# predict answer
pred_answer = inference(model, test_dataset, device)
# make csv file with predicted answer
# 아래 directory와 columns의 형태는 지켜주시기 바랍니다.

output = pd.DataFrame(pred_answer, columns=['pred'])
output.to_csv('submission_la.csv', index=False)

print('done')

We need to remove 84 to truncate the inputbut the first sequence has a length 16. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 21 to truncate the inputbut the first sequence has a length 10. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 43 to truncate the inputbut the first sequence has a length 8. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 12 to truncate the inputbut the first sequence has a length 10. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.
We need to remove 51 to truncate the inputbut the first sequence has a length 9. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance '

done


In [8]:
print('전체 용량')
!df -h

print('\n\n내가 사용중인 용량')
!du -h --max-depth=1 | sort -hr


import torch, sklearn, gc
print('\ntorch 캐시 삭제')
torch.cuda.empty_cache()
print('sklearn 캐시 삭제')
#sklearn.datasets.clear_data_home()
print('GC 가동')
gc.collect()
print('휴지통 삭제')
!cd ~
!rm -r ./.local/share/Trash/files

print('\n전체 용량')
!df -h

print('\n\n내가 사용중인 용량')
!du -h --max-depth=1 | sort -hr

전체 용량
Filesystem      Size  Used Avail Use% Mounted on
overlay          99G   93G  1.3G  99% /
tmpfs            64M     0   64M   0% /dev
tmpfs            30G     0   30G   0% /sys/fs/cgroup
shm             1.0G  4.0K  1.0G   1% /dev/shm
/dev/xvdb1       99G   93G  1.3G  99% /etc/hosts
tmpfs            30G   12K   30G   1% /proc/driver/nvidia
/dev/xvda1       48G  9.1G   36G  21% /usr/bin/nvidia-smi
udev             30G     0   30G   0% /dev/nvidia0
tmpfs            30G     0   30G   0% /proc/acpi
tmpfs            30G     0   30G   0% /proc/scsi
tmpfs            30G     0   30G   0% /sys/firmware


내가 사용중인 용량
45G	.
21G	./.local
13G	./unlabeled
7.4G	./unlabeled_128
3.4G	./.cache
679M	./kobert
150M	./input
15M	./Dacon
3.6M	./.ipython
312K	./.ipynb_checkpoints
288K	./logs
64K	./hard_vote
52K	./code
20K	./.jupyter
8.0K	./__pycache__
8.0K	./.nv
8.0K	./.config
8.0K	./.conda
4.0K	./.empty

torch 캐시 삭제
sklearn 캐시 삭제
GC 가동
휴지통 삭제

전체 용량
Filesystem      Size  Used Avail Use% Mounted on
overlay  

In [35]:
!du -hs

5.1G	.
